Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 3 <br>
Date: 11/20/2021 <br>

You are to apply the following techniques to mine this data set:

- Association Rules
- Decision tree Classification
- Naive Bayes Classification
- kmeans and dbscan clustering

Also, you are to use Principle Component Analysis to reduce the number of attributes (aka features)

## 1. What can you deduce from the data set? (in otherwords, what attribute values are indicative of "success")

### TODO ANSWER GERE
## 2. Which mining techniques yielded the best results for what? How do you define "best results"?
### TODO ANSWER GERE

### 3. How useful was Principle Component Analysis? For what number of components did you get the "best results."
### TODO ANSWER GERE
PCA as alright. There was a variety of different answers. There was an even spread and no noticable
dip that made it easy to just cut off the data. It was useful for DBSCAN and K-means Clustering to aid in the
visualization

Please include tables and/or graphs to justify your statements about which are best.

### Start of The Code
Main Library Imports Below

In [15]:
import graphviz
import pandas as pd
import numpy as np
from csv import reader

# Sklearn Libraries
from sklearn import tree
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import *
from sklearn.naive_bayes import *
from sklearn.cluster import KMeans, DBSCAN
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.tree import export_graphviz, export_text
import pydotplus

# Importing mlxtend Libraries
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Graphing Libraries
import matplotlib.pyplot as plt
import seaborn as sns

Importing the main document <br>
Setting the random seed and setting view options <br>

In [16]:
np.random.seed(0)
df =  pd.read_csv("bank-full.csv")

pd.set_option('display.max_columns', None)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.options.display.width = 0   # automatically ajust to window length

df_a = df.copy()
df_b = df.copy()

In [17]:
df_a.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### Association Rule Analysis Below
### Feature Engineering
First Cleaning up the data and creating categories

In [18]:
for index, row in df_a.iterrows():
	if row['balance'] < 0:
		row['balance'] = 'inDebt'
	else:
		row['balance'] = 'notInDebt'

# Creating Categories for balance
df_a.loc[(df_a['balance'] >= 10000), 'balanceSummary'] = 'veryPositive'
df_a.loc[((df_a['balance'] < 0) & (df_a['balance'] >= -10000)), 'balanceSummary'] = 'negative'
df_a.loc[((df_a['balance'] < 10000) & (df_a['balance'] >= 0)), 'balanceSummary'] = 'positive'
df_a.loc[df_a['balance'] < -500, 'balanceSummary'] = 'veryNegative'

# Creating Categories for ages
df_a.loc[((df_a['age'] < 25) & (df_a['age'] >= 0)),  'ageBand'] = 'ageBand1'
df_a.loc[((df_a['age'] < 30) & (df_a['age'] >= 25)), 'ageBand'] = 'ageBand2'
df_a.loc[((df_a['age'] < 40) & (df_a['age'] >= 30)), 'ageBand'] = 'ageBand3'
df_a.loc[((df_a['age'] < 50) & (df_a['age'] >= 40)), 'ageBand'] = 'ageBand4'
df_a.loc[((df_a['age'] < 120) & (df_a['age'] >= 50)),'ageBand'] = 'ageBand5'

# Creating Categories for default
df_a.loc[(df_a['default'] == 'no'), 'defaultValue'] = 'defaultNo'
df_a.loc[(df_a['default'] == 'yes'), 'defaultValue'] = 'defaultYes'

# Creating Categories for Housing
df_a.loc[(df_a['housing'] == 'no'), 'housingVal'] = 'houseNo'
df_a.loc[(df_a['housing'] == 'yes'), 'housingVal'] = 'houseYes'

# Creating Categories for Loans
df_a.loc[(df_a['loan'] == 'no'), 'loanVal'] = 'loanNo'
df_a.loc[(df_a['loan'] == 'yes'), 'loanVal'] = 'loanYes'

# Dropping all values that are not strings
df_a = df_a.drop(['age', 'day', 'duration', 'balance', 'campaign', 'previous', 'default', 'housing', 'loan', 'pdays'],
                 axis=1)

# Displaying Head
df_a.head()

,job,marital,education,contact,month,poutcome,y,balanceSummary,ageBand,defaultValue,housingVal,loanVal
0,management,married,tertiary,unknown,may,unknown,no,positive,ageBand5,defaultNo,houseYes,loanNo
1,technician,single,secondary,unknown,may,unknown,no,positive,ageBand4,defaultNo,houseYes,loanNo
2,entrepreneur,married,secondary,unknown,may,unknown,no,positive,ageBand3,defaultNo,houseYes,loanYes
3,blue-collar,married,unknown,unknown,may,unknown,no,positive,ageBand4,defaultNo,houseYes,loanNo
4,unknown,single,unknown,unknown,may,unknown,no,positive,ageBand3,defaultNo,houseNo,loanNo


Displaying all the Unique Values

In [19]:
for col in df_a:
    print(col, ":", df_a[col].unique())

job : ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital : ['married' 'single' 'divorced']
education : ['tertiary' 'secondary' 'unknown' 'primary']
contact : ['unknown' 'cellular' 'telephone']
month : ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
poutcome : ['unknown' 'failure' 'other' 'success']
y : ['no' 'yes']
balanceSummary : ['positive' 'negative' 'veryPositive' 'veryNegative']
ageBand : ['ageBand5' 'ageBand4' 'ageBand3' 'ageBand2' 'ageBand1']
defaultValue : ['defaultNo' 'defaultYes']
housingVal : ['houseYes' 'houseNo']
loanVal : ['loanNo' 'loanYes']


Exporting the dataframe as a CSV and reimporting it to speed up run times.

In [20]:
df_a.to_csv("outfile.csv", index=False)
file = []
with open("outfile.csv", 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        file.append(row)

Creating transact 1 hot boolean encoded numpy array and displaying dataframe head

In [21]:
te = TransactionEncoder()
te_ary = te.fit(file).transform(file)
df_a_encoded = pd.DataFrame(te_ary, columns=te.columns_)
df_a_encoded.head()

,admin.,ageBand,ageBand1,ageBand2,ageBand3,ageBand4,ageBand5,apr,aug,balanceSummary,blue-collar,cellular,contact,dec,defaultNo,defaultValue,defaultYes,divorced,education,entrepreneur,failure,feb,houseNo,houseYes,housemaid,housingVal,jan,job,jul,jun,loanNo,loanVal,loanYes,management,mar,marital,married,may,month,negative,no,nov,oct,other,positive,poutcome,primary,retired,secondary,self-employed,sep,services,single,student,success,technician,telephone,tertiary,unemployed,unknown,veryNegative,veryPositive,y,yes
0,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,True,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False


Creating Filter Function for frequent Item Sets

In [22]:
def filter_freq_items(freq_items,items=1):
    """This function filters the frequent items so that we can look a certain n-itemsets"""
    filtered_freq_items = freq_items[freq_items["length"] == items].copy()
    return filtered_freq_items

Creating Frequent Items List and Association Rules

In [25]:
# Creating Frequent Items List
freq_items = apriori(df_a_encoded, min_support=.5, use_colnames=True)
freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))
freq_items.sort_values(["support", "length"], inplace=True, ascending=False)
freq_items.head(10)

,support,itemsets,length
1,0.981952,(defaultNo),1
6,0.898346,(positive),1
17,0.890073,"(defaultNo, positive)",2
5,0.882996,(no),1
16,0.866120,"(defaultNo, no)",2
3,0.839755,(loanNo),1
14,0.828386,"(defaultNo, loanNo)",2
8,0.826528,(unknown),1
19,0.809741,"(defaultNo, unknown)",2
27,0.788994,"(no, positive)",2


In [26]:
# Creating Association Rules
rules = association_rules(freq_items, metric="confidence", min_threshold=0.3)
rules.sort_values(["lift"], inplace=True, ascending=False)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
233,(houseYes),"(defaultNo, no)",0.555826,0.866120,0.504048,0.906844,1.047020,0.022636,1.437170
228,"(defaultNo, no)",(houseYes),0.866120,0.555826,0.504048,0.581961,1.047020,0.022636,1.062518
227,(no),(houseYes),0.882996,0.555826,0.513028,0.581008,1.045306,0.022236,1.060102
226,(houseYes),(no),0.555826,0.882996,0.513028,0.923000,1.045306,0.022236,1.519545
232,(no),"(defaultNo, houseYes)",0.882996,0.546205,0.504048,0.570838,1.045100,0.021751,1.057399
229,"(defaultNo, houseYes)",(no),0.546205,0.882996,0.504048,0.922818,1.045100,0.021751,1.515961
176,"(defaultNo, no, loanNo)","(unknown, positive)",0.722928,0.738830,0.557463,0.771118,1.043701,0.023342,1.141068
177,"(unknown, positive)","(defaultNo, no, loanNo)",0.738830,0.722928,0.557463,0.754520,1.043701,0.023342,1.128699
185,"(no, loanNo)","(defaultNo, unknown, positive)",0.733478,0.731045,0.557463,0.760027,1.039644,0.021257,1.120770
168,"(defaultNo, unknown, positive)","(no, loanNo)",0.731045,0.733478,0.557463,0.762556,1.039644,0.021257,1.122463


### Association rules analysis
Below are different tables that show the break down of the frequent item sets
#### Frequent Items

In [33]:
filter_freq_items(freq_items,5)

,support,itemsets,length
51,0.557463,"(positive, unknown, no, defaultNo, loanNo)",5


In [34]:
filter_freq_items(freq_items,4)

,support,itemsets,length
46,0.660444,"(defaultNo, no, positive, loanNo)",4
49,0.659692,"(defaultNo, unknown, no, positive)",4
48,0.622180,"(defaultNo, unknown, positive, loanNo)",4
47,0.611298,"(defaultNo, unknown, no, loanNo)",4
50,0.562107,"(unknown, no, positive, loanNo)",4


In [38]:
filter_freq_items(freq_items,3).head(10)

,support,itemsets,length
39,0.781253,"(defaultNo, no, positive)",3
35,0.760440,"(defaultNo, positive, loanNo)",3
40,0.733323,"(defaultNo, unknown, no)",3
41,0.731045,"(defaultNo, unknown, positive)",3
34,0.722928,"(defaultNo, no, loanNo)",3
36,0.679842,"(defaultNo, unknown, loanNo)",3
45,0.666991,"(unknown, no, positive)",3
42,0.665399,"(no, positive, loanNo)",3
44,0.627157,"(unknown, positive, loanNo)",3
43,0.621096,"(unknown, no, loanNo)",3


In [37]:
filter_freq_items(freq_items,2).head(10)

,support,itemsets,length
17,0.890073,"(defaultNo, positive)",2
16,0.866120,"(defaultNo, no)",2
14,0.828386,"(defaultNo, loanNo)",2
19,0.809741,"(defaultNo, unknown)",2
27,0.788994,"(no, positive)",2
22,0.765770,"(positive, loanNo)",2
28,0.749049,"(unknown, no)",2
29,0.738830,"(unknown, positive)",2
21,0.733478,"(no, loanNo)",2
23,0.690392,"(unknown, loanNo)",2


After breaking down the frequent item sets it is interesting to learn that there is only
1 frequent item set that contains 5 words with a support over 5. This will make our cluster analysis
a bit more interesting later. With that said.
Positive balance, unknown job, no default and no loan is not is not likely to subscribe
at least with a .557463 support.

It is also interesting to see that the majority of the frequent item sets contain "no" meaning
that they are not likely to subscribe. The majority of the banks customers had defaultno with a
positive balance summary, or were not likely to subscribe.

It is also interesting to see that yes is not included in the frequent item sets. meaning that
the majority of people that say yes do not have a whole lot in common with others that said yes

#### Association rules

In [53]:
rules.sort_values('lift', ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
233,(houseYes),"(defaultNo, no)",0.555826,0.866120,0.504048,0.906844,1.047020,0.022636,1.437170
228,"(defaultNo, no)",(houseYes),0.866120,0.555826,0.504048,0.581961,1.047020,0.022636,1.062518
227,(no),(houseYes),0.882996,0.555826,0.513028,0.581008,1.045306,0.022236,1.060102
226,(houseYes),(no),0.555826,0.882996,0.513028,0.923000,1.045306,0.022236,1.519545
232,(no),"(defaultNo, houseYes)",0.882996,0.546205,0.504048,0.570838,1.045100,0.021751,1.057399


The top 5 most interesting rules, defined by the highest lift values is that homeowners
are not likely to default and subscribe. This not exactly shocking to find out. It is intresting to see
that the lift values are not extremely high.

In [54]:
rules.sort_values('confidence', ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
20,"(positive, loanNo)",(defaultNo),0.765770,0.981952,0.760440,0.993039,1.011291,0.008490,2.592815
71,"(no, positive, loanNo)",(defaultNo),0.665399,0.981952,0.660444,0.992554,1.010797,0.007055,2.423952
107,"(unknown, positive, loanNo)",(defaultNo),0.627157,0.981952,0.622180,0.992065,1.010299,0.006343,2.274488
163,"(unknown, no, positive, loanNo)",(defaultNo),0.562107,0.981952,0.557463,0.991737,1.009965,0.005500,2.184189
210,"(married, positive)",(defaultNo),0.541206,0.981952,0.536694,0.991663,1.009890,0.005256,2.164823


Looking at the top five strong rules reveals th default no has the strongest confidence if
the antecedents are, loanNo, positive bank account  and are married or have an unknown job.

Unfortunately these word associations are not leading us to find anything that is helping find rules
that are indicative of a successful campaign. It is for that reason that I will continue data mining with the
remain algorithms.

### Feature Engineering for the following algorithms
- Decision tree Classification
- Naive Bayes Classification
- kmeans and dbscan clustering

First we must take all the data and numericalize it

In [55]:
# Creating Categories for ages then making categories
df_b['job'] = df_b['job'].astype('category')
df_b['job'] = df_b['job'].cat.codes
df_b['marital'] = df_b['marital'].astype('category')
df_b['marital'] = df_b['marital'].cat.codes
df_b['education'] = df_b['education'].astype('category')
df_b['education'] = df_b['education'].cat.codes
df_b['default'] = df_b['default'].astype('category')
df_b['default'] = df_b['default'].cat.codes
df_b['contact'] = df_b['contact'].astype('category')
df_b['contact'] = df_b['contact'].cat.codes
df_b['month'] = df_b['month'].astype('category')
df_b['month'] = df_b['month'].cat.codes
df_b['poutcome'] = df_b['poutcome'].astype('category')
df_b['poutcome'] = df_b['poutcome'].cat.codes
df_b['housing'] = df_b['housing'].astype('category')
df_b['housing'] = df_b['housing'].cat.codes
df_b['loan'] = df_b['loan'].astype('category')
df_b['loan'] = df_b['loan'].cat.codes
df_b['y'] = df_b['y'].astype('category')
df_b['y'] = df_b['y'].cat.codes
df_b = df_b.drop('pdays',axis=1)
df_b['balance'] = (df_b['balance'] - df_b['balance'].min()) / ( df_b['balance'].max() - df_b['balance'].min())
print("Post Conversion of Data into numeric, dropping pdays, and normalizing balance:")
df_b.head(10)

Post Conversion of Data into numeric, dropping pdays, and normalizing balance:


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,y
0,58,4,1,2,0,0.092259,1,0,2,5,8,261,1,0,3,0
1,44,9,2,1,0,0.073067,1,0,2,5,8,151,1,0,3,0
2,33,2,1,1,0,0.072822,1,1,2,5,8,76,1,0,3,0
3,47,1,1,3,0,0.086476,1,0,2,5,8,92,1,0,3,0
4,33,11,2,3,0,0.072812,0,0,2,5,8,198,1,0,3,0
5,35,4,1,2,0,0.074901,1,0,2,5,8,139,1,0,3,0
6,28,4,2,2,0,0.076862,1,1,2,5,8,217,1,0,3,0
7,42,2,0,2,1,0.072822,1,0,2,5,8,380,1,0,3,0
8,58,5,1,0,0,0.073902,1,0,2,5,8,50,1,0,3,0
9,43,9,2,1,0,0.078187,1,0,2,5,8,55,1,0,3,0
